In [558]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from arch import arch_model
import torch
import pytorch_lightning as pl 
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from pytorch_lightning.loggers import TensorBoardLogger

In [559]:


"""
должно быть omega, d , phi, beta

    # omega > 0 <- 1
    # 0 <= d <= 1 <- 2
    # 0 <= phi <= (1 - d) / 2 <- 2
    # 0 <= beta <= d + phi <- 2

""";


In [560]:
def generate_ground_garch(omega, d, phi, beta, n =3000):
    
    am = arch_model(None, mean='Zero', vol='FIGARCH', p=1, q=1, power = 2) #Остатки просто получаются умножением волатильности на кси ~N(0,1)
    params = np.array([omega, d, phi, beta])
    am_data = am.simulate(params, n)

    return am_data['data'].to_numpy(), am_data['volatility'].to_numpy()

In [561]:
omega = 0.1
d =  0.2
phi = 0.2
beta = 0.2

In [562]:
data, volat = generate_ground_garch(omega, d, phi ,beta)

In [563]:
truncation_size = 10

In [564]:
np.std(volat)

0.1494428635649965

In [565]:
class FIGARCHDataset(Dataset):
    def __init__(self, residuals, volatility, T, scale = 100 ):
        super().__init__()
        self.rsquared = torch.FloatTensor(np.square(residuals) * scale)
        self.vol = torch.FloatTensor(volatility * scale)
        self.T = T
        self.valid_index = range(self.T-1, len(self.rsquared))

    def __len__ (self):
        return len(self.valid_index)
    

    def __getitem__(self, idx):
        t = self.valid_index[idx]
        return (
            self.rsquared[t-self.T+1: t+1],
            self.vol[t],
            self.rsquared[t]
        )



In [566]:
ds = FIGARCHDataset(data, volat, truncation_size)

In [567]:
len(data)

3000

In [568]:
print(ds[0])

(tensor([235.9222, 212.9424, 307.9615, 271.1602, 476.0608, 103.4897, 158.1664,
         19.3226,   1.1775,  32.3695]), tensor(82.7130), tensor(32.3695))


In [569]:
np.square(data[0])*100, volat[10]*100

(235.92217071586492, 81.07701246700896)

In [570]:
weight_list = []

In [571]:
class FIGARCHModule (pl.LightningDataModule):
    def __init__(self, data, volat, truncation_size, batch_size=32):
        super().__init__()
        self.data = data
        self.volat = volat
        self.truncation_size = truncation_size
        self.batch_size = batch_size

    def setup(self, stage = None):
        self.train_dataset = FIGARCHDataset(self.data,self.volat, self.truncation_size)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, self.batch_size, shuffle=False)

In [572]:
class CorrectedNLoss(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, pred_var, target_eps_squared):

        loss =  (0.5*(torch.log(pred_var)) + (target_eps_squared/(2*pred_var))) 
        return loss.mean()

In [573]:
class FIGARCH_CNN(pl.LightningModule):
    def __init__(self, truncation_size, lr = 1e-3):
        super().__init__()
        self.truncation_size = truncation_size
        self.conv = nn.Conv1d(1,1, kernel_size=truncation_size, bias= False)
        self.criterion = CorrectedNLoss()
        self.lr = lr

        with torch.no_grad():
            weights = torch.full((1, 1, truncation_size), 0.1)
            self.conv.weight.copy_(weights)
        
    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.conv(x)
        x = x.squeeze(2)

        return x.squeeze(1)
    
    def training_step(self, batch):
        rsq_window, vol, rsq_val = batch
        pred_var = self.forward(rsq_window)
        loss = self.criterion(pred_var, rsq_val)
        self.log('train_loss', loss)

        return loss
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)
    
    def on_train_end(self):
        weight_list.clear()
        weights = self.conv.weight.detach().cpu().numpy().squeeze()
        weight_list.append(weights)


In [574]:
dm = FIGARCHModule(data,volat, truncation_size, batch_size= 32)
model = FIGARCH_CNN(truncation_size)

In [575]:
logger = TensorBoardLogger('tb_logs', name = 'figarch_model')

In [ ]:
trainer = pl.Trainer(
    max_epochs=10,
    accelerator='auto',
    logger = logger
)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [577]:
trainer.fit(model, dm)


  | Name      | Type           | Params | Mode 
-----------------------------------------------------
0 | conv      | Conv1d         | 10     | train
1 | criterion | CorrectedNLoss | 0      | train
-----------------------------------------------------
10        Trainable params
0         Non-trainable params
10        Total params
0.000     Total estimated model params size (MB)
2         Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.


In [578]:
weight_list

[array([0.0337276 , 0.0368989 , 0.00414068, 0.01994677, 0.02279804,
        0.01277062, 0.00421026, 0.01394118, 0.02735223, 0.700993  ],
       dtype=float32)]

In [579]:
def decompute_lambdas(lambdas):
    

SyntaxError: incomplete input (3251068782.py, line 2)